In [ ]:
%pip install openai

In [1]:
from config import api_key
from openai import OpenAI
client = OpenAI(api_key = api_key)
client

In [8]:
uploaded_file = client.files.create(
    file=open("story.txt",'rb'),
    purpose='assistants'
)

In [9]:
uploaded_file

FileObject(id='file-lLifbxWxZCkPiDVu7SscVIql', bytes=5299, created_at=1731125620, filename='story.txt', object='file', purpose='assistants', status='processed', status_details=None)

In [17]:
uploaded_file.id

'file-lLifbxWxZCkPiDVu7SscVIql'

In [22]:
!Pwd

/Users/mtxit/Desktop/self/mlops/llm_practice/1.llmops-project-1-chatgpt-docker


In [2]:
# Create a vector store caled "Story teller"
vector_store = client.beta.vector_stores.create(name="Story Teller")
 
# Ready the files for upload to OpenAI
file_paths = ["/Users/mtxit/Desktop/self/mlops/llm_practice/1.llmops-project-1-chatgpt-docker/story.txt"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

In [4]:
assistant = client.beta.assistants.create(
    instructions="You are a personal math tutor. When asked a math question, write and run code to answer the question.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],  
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},

)

In [5]:
thread = client.beta.threads.create()
thread

Thread(id='thread_JXIvQTU46pI9YYOOcJFjItSz', created_at=1731127301, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [6]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Who is the hero of the story?"
)

In [7]:
message

Message(id='msg_8CmNldwdRmJFp6ZsewXND3jF', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who is the hero of the story?'), type='text')], created_at=1731127303, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_JXIvQTU46pI9YYOOcJFjItSz')

In [8]:
assistant

Assistant(id='asst_4tE0dB1b0XGX6uMeAG3Ly7nm', created_at=1731127300, description=None, instructions='You are a personal math tutor. When asked a math question, write and run code to answer the question.', metadata={}, model='gpt-4o', name=None, object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_syqAzkjcra03tYooMDr7e59H'])), top_p=1.0)

In [9]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [11]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
run.status

'completed'

In [12]:
message

Message(id='msg_8CmNldwdRmJFp6ZsewXND3jF', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who is the hero of the story?'), type='text')], created_at=1731127303, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_JXIvQTU46pI9YYOOcJFjItSz')

In [13]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
messages

SyncCursorPage[Message](data=[Message(id='msg_pUc1AK62TdpmAqG7Ey13hu0H', assistant_id='asst_4tE0dB1b0XGX6uMeAG3Ly7nm', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=185, file_citation=FileCitation(file_id='file-PqiUmZ6KyVsokzGyVYrM9fRT'), start_index=170, text='【4:0†story.txt】', type='file_citation')], value='The hero of the story is Dr. Michael Greene, a brilliant inventor who creates a revolutionary product called "Lumina" designed to enhance human potential and productivity【4:0†story.txt】.'), type='text')], created_at=1731127308, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_AgQFWyYHB8kZl46kjvOZlGne', status=None, thread_id='thread_JXIvQTU46pI9YYOOcJFjItSz'), Message(id='msg_8CmNldwdRmJFp6ZsewXND3jF', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who is the hero of the story?'), ty

In [15]:
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run.status=="completed":
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        print(text)
        break

The hero of the story is Dr. Michael Greene, a brilliant inventor who creates a revolutionary product called "Lumina" designed to enhance human potential and productivity【4:0†story.txt】.
